In [1]:
import os
import os.path as osp
import argparse
from cv2 import imread, normalize, resize
import cv2
import json
import numpy as np
import time
import datetime
import random
import logging
import sys
from PIL import Image
from pytorchtools import EarlyStopping # 위 링크의 깃허브 파일에서 임포트
from utils import euclidean_metric, one_hot, count_acc
from utils import pprint, set_gpu, ensure_path, AverageMeter, Timer, accuracy, one_hot

import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch import nn, optim, autograd
from torch.utils.data import DataLoader
import torch.utils.data as data
import torchvision
from random import sample, random

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset, Dataset, TensorDataset, WeightedRandomSampler
import os.path as osp
from tqdm import tqdm

from random import sample, random

import clip
import skimage
import IPython.display
import matplotlib.pyplot as plt

from collections import OrderedDict
import torch

batch_size = 64
num_epoch = 100

/home/iai/anaconda3/envs/sh_clip/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

zero_shot_model, zero_shot_preprocess= clip.load("ViT-B/32", device=device)

# Freeze the model
#for param in zero_shot_model.parameters():
#    param.requires_grad = False
    
fine_tunning_model ,fine_tunning_preprocess= clip.load("ViT-B/32", device=device)

In [3]:
class Customdataset(data.Dataset):
    def __init__(self,transform = None):
        dalist=None
        self.file_paths = []
        self.label = []
        self.concept = []
        self.transform = transform
        
        context_list = ['cartoon', 'art_painting', 'sketch']
        label_list = os.listdir('/home/iai/Desktop/SH/Data/PACS/art_painting')
        
        word2index = {}
        for voca in label_list:
            if voca not in word2index.keys():
                word2index[voca] = len(word2index)
        print(word2index)
                  
        
        for context in context_list:
            for label in label_list:
                img_list = os.listdir('/home/iai/Desktop/SH/Data/PACS/' + context +'/' + label)
                #self.file_paths = self.file_paths + img_list
                
                for img in img_list:
                    self.file_paths.append('/home/iai/Desktop/SH/Data/PACS/' + context +'/' + label +'/'+img)
                
                #tmp_list_label = [self.one_hot_encoding(label, word2index)] * len(img_list)
                tmp_list_label = [word2index[label]] * len(img_list)
                self.label = self.label + tmp_list_label
                
                tmp_list_context = [context] * len(img_list)
                self.concept = self.concept + tmp_list_context
            
        print(f'len image :{len(self.file_paths)}')      
        print(f'len label :{len(self.label)}')      
        print(f'len concept :{len(self.concept)}') 
        
        print(self.label[0])
    '''
    def one_hot_encoding(self, word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index] = 1
       
       return index
    '''
    def __len__(self):

        return len(self.file_paths)


    def __getitem__(self, idx):
                
        label = self.label[idx]
        path = self.file_paths[idx]
        image = Image.open(path).convert('RGB')
        concept = self.concept[idx]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label, concept

In [4]:
data_transforms = transforms.Compose([
        
        transforms.Resize(size=224),
        transforms.CenterCrop(size=224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),

        ])

##커스텀 데이터셋
dataset = Customdataset(transform=data_transforms)
#dataset = Customdataset(transform=fine_tunning_preprocess)
train_size = int(0.8 * len(dataset))
print(f"train size :{train_size}")
val_size = len(dataset) - train_size
print(f"test_size :{val_size}")
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"train_dataset :{len(train_dataset)}")
print(f'test dataset :{len(val_dataset)}')


train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size = batch_size, #배치사이즈
                                               shuffle = True 
                                               )

val_loader = torch.utils.data.DataLoader(val_dataset,
                                               batch_size = batch_size,                                              
                                               shuffle = True 
                                              )

{'giraffe': 0, 'elephant': 1, 'horse': 2, 'person': 3, 'dog': 4, 'house': 5, 'guitar': 6}
len image :8321
len label :8321
len concept :8321
0
train size :6656
test_size :1665
train_dataset :6656
test dataset :1665


In [5]:
import torch.distributions as distributions

def custom_loss(zero_shot_output_context, fine_output_context, fine_output_label, target, alpha):

    print(fine_output_context.log_softmax(dim=-1).shape)
    #kl_loss = F.kl_div(fine_output_context.log_softmax(dim=-1), zero_shot_output_context.softmax(dim=-1), reduction='batchmean')
    critetion = torch.nn.KLDivLoss(reduction='batchmean')
    kl_loss = critetion(fine_output_context.log_softmax(dim=-1), zero_shot_output_context.softmax(dim=-1))
    print(kl_loss)
    
    
    print('qwer')
    ce_loss = F.cross_entropy(fine_output_label.softmax(dim=-1), target)
    print(ce_loss)

    
    final_loss = (ce_loss + alpha * kl_loss)
    print(final_loss)
    
    return final_loss

def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float()

In [6]:
a = torch.tensor([[0.9927937 , 0.00421068 ,0.00299572]])
a.squeeze(dim=0)
a

tensor([[0.9928, 0.0042, 0.0030]])

In [7]:
model_path = '/home/iai/Desktop/SH/cognex/model/'
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, fine_tunning_model.parameters()), lr= 5e-8, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
optimizer2 = torch.optim.AdamW(filter(lambda p: p.requires_grad, fine_tunning_model.parameters()), lr= 5e-8, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100)

early_stopping = EarlyStopping(patience = 4, verbose = True, min_epoch=10)

global_count = 0

train_loss_arr =[]
val_loss_arr =[]
n = len(train_loader)

context_list = ['cartoon', 'art_painting', 'sketch']
label_list = os.listdir('/home/iai/Desktop/SH/Data/PACS/art_painting')

label_text = []
for label in label_list:
    label_text.append("a photo of " + str(label))
    
context_label_text = []
for label in label_list:
    for context in context_list:
        context_label_text.append("a " +str(context) + " of " + str(label))

label_text = clip.tokenize(label_text).to(device)
context_label_text = clip.tokenize(context_label_text).to(device)


torch.autograd.set_detect_anomaly(True)
for epoch in range(1, num_epoch + 1):
    
    fine_tunning_model.train()
    train_running_loss = 0.0
    train_running_accuracy = 0.0
    total = 0 
       
    for i,[image,label, context] in enumerate(train_loader):
        
        target = label.to(device)
        input_image = image.to(device)
        
        with torch.no_grad():            
            logits_per_image, logits_per_text = zero_shot_model(input_image, context_label_text)
            zero_shot_output_context = logits_per_image.float()
        
        #fine_tunning_image_features = fine_tunning_model.encode_image(input_image).float()
        #fine_tunning_context_text_features = fine_tunning_model.encode_text(context_label_text).float()
        
        #fine_tunning_image_features /= fine_tunning_image_features.norm(dim=-1, keepdim=True)
        #fine_tunning_context_text_features /= fine_tunning_context_text_features.norm(dim=-1, keepdim=True).requires_grad_(True)
        #fine_tunning_output_context = (fine_tunning_image_features @ fine_tunning_context_text_features.T)
        logits_per_image, logits_per_text = fine_tunning_model(input_image, context_label_text)
        fine_tunning_output_context = logits_per_image.float()
        
        
        #fine_tunning_label_text_features = fine_tunning_model.encode_text(label_text).float()
        #fine_tunning_label_text_features /= fine_tunning_label_text_features.norm(dim=-1, keepdim=True).requires_grad_(True).float()
        #fine_tunning_output_label = (fine_tunning_image_features @ fine_tunning_label_text_features.T)
        logits_per_image, logits_per_text = fine_tunning_model(input_image, label_text)
        fine_tunning_output_label = logits_per_image.float()
        
        
        loss = custom_loss(zero_shot_output_context, fine_tunning_output_context, fine_tunning_output_label, target, alpha=1e0)

        optimizer.zero_grad()
        optimizer2.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        '''
        if device == "cpu":
            optimizer.step()
        else : 
            #convert_models_to_fp32(fine_tunning_model)
            optimizer.step()
            clip.fine_tunning_model.convert_weights(fine_tunning_model)
        '''    
        torch.cuda.synchronize()
        train_running_loss += loss.item()
        
        total += target.size(0)
        _, predicted = torch.max(fine_tunning_output_label, 1) 
        train_running_accuracy += (predicted ==target).sum().item() 
        
        
    train_loss_arr.append(train_running_loss / n)
    train_accuracy = (100 * train_running_accuracy / total)
    lr_scheduler.step()      
    
    fine_tunning_model.eval()
    val_running_loss = 0.0
    val_running_accuracy = 0.0
    total = 0 
    
    with torch.no_grad():
        for j,[image,label, context] in enumerate(val_loader):
            
            target = label.to(device)
            input_image = image.to(device)
 
            logits_per_image, logits_per_text = zero_shot_model(input_image, context_label_text)
            zero_shot_output_context = logits_per_image
        
            logits_per_image, logits_per_text = fine_tunning_model(input_image, context_label_text)
            fine_tunning_output_context = logits_per_image

            logits_per_image, logits_per_text = fine_tunning_model(input_image, label_text)
            fine_tunning_output_label = logits_per_image

            loss = custom_loss(zero_shot_output_context, fine_tunning_output_context, fine_tunning_output_label, target, alpha=1e0)
            
            total += target.size(0)
            val_running_loss += loss.item()
            _, predicted = torch.max(fine_tunning_output_label, 1) 
            val_running_accuracy += (predicted == target).sum().item() 
        
        val_loss_arr.append(val_running_loss / len(val_loader))       
        early_stopping(val_running_loss, fine_tunning_model)
        torch.save(fine_tunning_model, model_path + str(epoch-1) + 'epoch_model.pt')
        
        val_accuracy = (100 * val_running_accuracy / total)  

        print('Epoch: %d , train_loss: %.5f  , train_accuracy: %.2f , val_loss: %.5f , val_accuracy: {%.2f}' 
            %(epoch , train_running_loss / len(train_loader) ,train_accuracy , val_running_loss / len(val_loader) , val_accuracy))

        
        if early_stopping.early_stop: # 조건 만족 시 조기 종료
            break


torch.Size([64, 21])
tensor(-1.0099e-10, device='cuda:0', grad_fn=<DivBackward0>)
qwer
tensor(1.2750, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2750, device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([64, 21])
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
qwer
tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)


[W python_anomaly_mode.cpp:104] Warning: Error detected in LogSoftmaxBackward. Traceback of forward call that caused the error:
  File "/home/iai/anaconda3/envs/sh_clip/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/iai/anaconda3/envs/sh_clip/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/iai/anaconda3/envs/sh_clip/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/iai/anaconda3/envs/sh_clip/lib/python3.8/site-packages/traitlets/config/application.py", line 1041, in launch_instance
    app.start()
  File "/home/iai/anaconda3/envs/sh_clip/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 711, in start
    self.io_loop.start()
  File "/home/iai/anaconda3/envs/sh_clip/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/home/iai/anacond

RuntimeError: Function 'LogSoftmaxBackward' returned nan values in its 0th output.

In [ ]:
import matplotlib.pyplot as plt


plt.plot(train_loss_arr,
         color='skyblue',
         marker='o', markerfacecolor='blue',
         markersize=4)

plt.plot(val_loss_arr,
         color='pink',
         marker='x', markerfacecolor='red',
         markersize=4)

plt.title('Model loss')
plt.xlabel('epoch')
plt.show()